# Building a new universal model for prokaryotes from the BIGG universe

Two models: gram-negative and gram-positive. Gram-negative contains: extracellular, periplasm and cytoplasm. Gram-positive only extracellular and cytoplasm.

1) Gran-negative, take all reactions that are not in those compartments and move them to cytoplasm. Remove shuttle/transport reactions between compartments that are not in gram neg

2) Remove duplicated reactions/metabolites in the model

In [1]:
import cobra
from cobra import Model

from reconstruction import remove_shuttle_reactions, move_reactions_to_cytoplasm


def test_remove_shuttle_reactions(model: Model, allowed_compartments: set = {}) -> bool:
    """"""
    return True if not [
        rxn for rxn in model.reactions if not rxn.compartments.issubset(allowed_compartments)
        ] else False

In [2]:
uniprok = cobra.io.read_sbml_model("../data/carveme_universes/BIGG_universal_model/universal_model_cobrapy.xml")
uniprok

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-05


Name,bigg_universal
Memory address,7f4969acc910
Number of metabolites,15638
Number of reactions,28301
Number of genes,0
Number of groups,0
Objective expression,1.0*BIOMASS_reaction - 1.0*BIOMASS_reaction_reverse_5a818
Compartments,"cytoplasm, extracellular, periplasm, mitochondrion, peroxisome, unknown, nucleus, vacuole, golgi, thylakoid, lysosome, chloroplast, eyespot, flagellum, mitochondrial intermembrane space, unknown, unknown, unknown, unknown, mitochondrial membrane, cell wall, unknown"


## Remove unwanted shuttle reactions

In [3]:
prok_compartments = {"e", "c", "p"}
uniprok = remove_shuttle_reactions(uniprok, allowed_compartments=prok_compartments)
uniprok

Name,bigg_universal
Memory address,7f4969acc910
Number of metabolites,15467
Number of reactions,25787
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,"cytoplasm, extracellular, periplasm, mitochondrion, peroxisome, unknown, nucleus, vacuole, golgi, thylakoid, lysosome, chloroplast, eyespot, flagellum, unknown, unknown, cell wall, unknown"


## Move reactions to cytoplasm

In [4]:
uniprok = move_reactions_to_cytoplasm(uniprok)
uniprok

Name,bigg_universal
Memory address,7f4969acc910
Number of metabolites,11970
Number of reactions,25787
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,"cytoplasm, extracellular, periplasm"


In [5]:
# Test if any reaction remains in illegal comparments
print("passed" if test_remove_shuttle_reactions(uniprok, prok_compartments) else "failed")

passed


## Annotate metabolites

In [6]:
from reconstruction import annotate_compounds


cpd_annotations = "/home/robaina/Documents/NewAtlantis/phycosphere/data/carveme_universes/mnx_compounds.tsv"
uniprok = annotate_compounds(uniprok, cpd_annotations)

## Add biomass reaction to universal gram negative model

Take reaction from universal CarveME gram negative model

In [8]:
carveme_gramneg = cobra.io.read_sbml_model("../data/carveme_universes/universe_bacteria.xml")

In [10]:
growth = carveme_gramneg.reactions.get_by_id("Growth")
uniprok.add_reactions([growth])
uniprok.objective = "Growth"
uniprok

Ignoring reaction 'Growth' since it already exists.


Name,bigg_universal
Memory address,7f4969acc910
Number of metabolites,11970
Number of reactions,25788
Number of genes,0
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytoplasm, extracellular, periplasm"


## Curate model

## Test model

In [11]:
def open_exchanges(model: Model) -> Model:
    """
    Open all exchanges in a model.

    Args:
        model (Model): a cobra model object

    Returns:
        Model: a cobra model object with all exchanges open
    """
    for exchange in model.exchanges:
        exchange.lower_bound = -1000
    return model

uniprok = open_exchanges(uniprok)

In [13]:
uniprok.slim_optimize()

575.5120866080945

## Write universal model

In [14]:
cobra.io.write_sbml_model(uniprok, "../data/carveme_universes/universal_prokaryote.xml")